# Distribution Strategies

### Introduction

In the last lesson, we saw how redshift partitions table rows across the slices of one or more compute nodes in a cluster.  In this lesson, we'll learn about different options in determining how how redshift distributes data among slices, and how these decisions can affect the performance of our queries.  

### Even Distribution

The default distribution strategy in redshift is the even distribution strategy.  With the even distribution, data is inserted in a round robin form, with the rows of data looping through each of the slices. 

We can see this strategy in the below diagram.  As we can see the first entry is inserted in Node 1, Slice 1, and then the second row of data is placed on Node 1, Slice 2, and so on.

<img src="./partitioned-data.png" width="100%">

* Even Distribution: The good

Now the benefit of the even distribution is that the data is evenly distributed amongst the slices.  All things equal, this is preferable to having all of our data stack up on a single slice, and in a single node, while other nodes sit idle.  

### When Even Distribution Goes Wrong

However, there is a downside to the even distribution.  In the diagram below, we have a fact table of movies, distributed across two nodess, and we also have a producers dimension table of the production companies.  Both the movies table and the producers table are distributed using even distribution. 

<img src="./exported.jpg" width="100%">

So now let's say that we want to find the name of company that produced the Dark Night.  To accomplish this, Redshift will have to connect data across two different nodes, which is a costly operation that results from *shuffling*.  

Shuffling is common with the even distribution, as our data is distributed with a simple round robin format.

<img src="./dist_key_full.jpg" width="100%">

### All Distribution

One way of preventing shuffling is with the *all* distribution strategy.  Here, an entire dimension table is copied to each of the nodes.  This way, when looking for to join with the production table, the query will never need to load data from a separate node.

<img src="./all_dist.jpg" width="100%">

A downside of the all distribution strategy, of course, is the space that it consumes.  It's for this reason, that the all strategy is only recommended for typically smaller dimension tables, and not for larger fact tables.  In redshift, an all distribution strategy will work for a dimension table of size 2 - 3 million.

### Key Distribution

The other distribution strategy is the key distribution.  With the key distribution strategy, the rows are distributed based on the value of a specific column.  For example, below we can see that redshift placed all of the movies with an odd `prod_id` on node 1 and the movies with an even `prod_id` on node 2.  

If we also using a key distribution on the primary key of the producers table, then this ensures that the data is co-located.  This way we again avoid joining with tables located on different nodes.

<img src="./dist_key_full.jpg" width="100%">

One thing to try to prevent with the key distribution strategy is rows piling up on just a single node.  This can happen if there are hot spots in the data.  For example, in the example above, we can see a lot of movies made by Fox.  This leads to an uneven distribution of ouro movies -- with a lot of movies on Node 1, and few movies on Node 2.  This can result in an inefficient use of our resources.  

For this reason, it's good to check that data will be fairly balanced, before using a key distribution strategy.  For this reason, distributing based on a time series value is often a good choice to ensure somewhat even distribution.

### Review

So what are the best distribution strategies? 

1. Key Distribution

Most preferable is the key distribution strategy, as it colocates data with the join.   A key distribution should be avoided however, if it will result in a skewed distribution.  So it's important to check that there are not a lot of hot spots.

2. All distribution 

If key distribution does not work, then the next alternative is all distribution.  Of course the downside is that all distribution takes up space by replicating tables.  So all distribution should only be used with dimension tables under 2 - 3 million rows.

3. Even Distribution

Finally, if key distribution results in a highly skewed distribution, and dimension tables are too large for an all distribution, then the even distribution should be used.

### Resources

[Redshift Deep Dive](https://www.youtube.com/watch?t=578&v=iuQgZDs-W7A&feature=youtu.be&ab_channel=AWSOnlineTechTalks)